<a href="https://colab.research.google.com/github/bhuiyanmobasshir94/MultiCoNER/blob/main/notebooks/spark_nlp_multiconer_000_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-01-12 19:21:35--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-01-12 19:21:35--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-01-12 19:21:36--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:44

In [9]:
!pip install gdown
!gdown --id 1AaaFcJ0Zzxr-fnlhlbp8mQePU6o8dFvk
!unzip -u "./BN_DATASET.zip" -d "./data"
!rm -f "./BN_DATASET.zip"
!cp -r ./data/BN-Bangla/. ./data && rm -rf ./data/BN-Bangla

Downloading...
From: https://drive.google.com/uc?id=1AaaFcJ0Zzxr-fnlhlbp8mQePU6o8dFvk
To: /content/BN_DATASET.zip
100% 1.38M/1.38M [00:00<00:00, 90.3MB/s]
Archive:  ./BN_DATASET.zip
   creating: ./data/BN-Bangla/
  inflating: ./data/BN-Bangla/bn_dev.conll  
  inflating: ./data/BN-Bangla/bn_train.conll  


In [10]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [11]:
# if you have GPU
spark = sparknlp.start(gpu=True)
# spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.0
Apache Spark version:  3.0.3


In [12]:
with open("data/bn_train.conll") as f:
    c=f.read()

print (c[:200])

# id 5e80f5c9-1196-4906-826b-4cbdcaec7b6f	domain=train
২০১৮ _ _ O
এর _ _ O
সেরা _ _ O
(বর্ণানুক্রমিকভাবে _ _ O
তালিকাভুক্ত, _ _ O
র‍্যাঙ্ক _ _ O
করা _ _ O
হয়নি), _ _ O
এনপিআর _ _ B-CORP

# id ba55278


In [110]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, "./data/bn_train.conll")
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|২০১৮ এর সেরা (বর্...|[[document, 0, 71...|[[document, 0, 71...|[[token, 0, 3, ২০...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|১৮২৫ সালে, তিনি ব...|[[document, 0, 81...|[[document, 0, 81...|[[token, 0, 3, ১৮...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|তিনি মার্কো মারুল...|[[document, 0, 53...|[[document, 0, 53...|[[token, 0, 3, তি...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [111]:
test_data = CoNLL().readDataset(spark, "./data/bn_dev.conll")
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [24]:
training_data.count()

15300

In [32]:
test_data.count()

800

In [113]:
# we use BERT Tiny
# bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
bert_annotator = XlmRoBertaEmbeddings.pretrained()\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[OK!]


In [114]:
test_data = bert_annotator.transform(test_data)
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|[[word_embeddings...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|[[word_embeddings...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [117]:
# let's transform and save our test dataset for evaluation
test_data.write.parquet("data/test_withEmbeds.parquet")

In [118]:
test_data.select("bert.result","bert.embeddings",'label.result').show()

+--------------------+--------------------+--------------------+
|              result|          embeddings|              result|
+--------------------+--------------------+--------------------+
|[সমস্ত, বেতন, নিল...|[[-0.003867594, 0...|[O, O, O, O, O, O...|
|[সংস্করণ, ৩, প্রক...|[[0.05987771, 0.0...|[O, O, O, O, B-CW...|
|[একটি, পাইলট, (হা...|[[2.2256654E-4, 0...|[O, O, O, O, O, B...|
|[প্রাইমাভেরা, ঝুঁ...|[[0.022214545, 0....|[B-CW, O, O, O, O...|
|[১৮০০, -এর, দশকে,...|[[0.019855844, -0...|[O, O, O, B-PER, ...|
|[চারজন, বিজয়ী, ড...|[[0.09979668, 0.0...|[O, O, O, O, B-PE...|
|[এটি, ছিল, নাটকীয...|[[-0.013127094, 0...|[O, O, O, O, O, B...|
|[এর, মধ্যে, একটি,...|[[-0.03805488, 0....|[O, O, O, B-CORP,...|
|[এরলং, সুযোগ, সমা...|[[-0.038714547, 0...|[B-CW, O, O, O, O...|
|[যদিও, গির্জার, স...|[[-0.049409423, 0...|[O, O, O, O, B-PR...|
|[দ্য, ম্যাগনিফিসে...|[[-0.032771435, 0...|[B-CW, I-CW, I-CW...|
|[দীর্ঘ, সময়, ধরে...|[[0.016834566, 0....|[O, O, O, O, O, O...|
|[তা, সত্ত্বেও,, এ...|[[-

In [124]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setGraphFolder("/content/data")\
  .setMaxEpochs(5)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setTestDataset("/content/data/test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37241)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [121]:
ner_model = pipeline.fit(training_data)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37241)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    co

In [ ]:
!rm -f /root/annotator_logs/NerDLApproach_3f2872bcff8e.log

In [102]:
!ls -l /root/annotator_logs/

total 8
-rw-r--r-- 1 root root 5153 Jan 12 21:03 NerDLApproach_3f2872bcff8e.log


In [103]:
!cat /root/annotator_logs/NerDLApproach_3f2872bcff8e.log

Name of the selected graph: /content/data/blstm_13_128_128_220.pb
Training started - total epochs: 5 - lr: 0.001 - batch size: 32 - labels: 13 - chars: 219 - training examples: 15300


Epoch 1/5 started, lr: 0.001, dataset size: 15300


Epoch 1/5 - 91.63s - loss: 3994.1873 - batches: 480
Quality on test dataset: 
time to finish evaluation: 2.08s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 33	 130	 68	 0.20245399	 0.32673267	 0.25
B-PROD	 1	 2	 189	 0.33333334	 0.005263158	 0.010362695
B-CW	 2	 19	 118	 0.0952381	 0.016666668	 0.028368797
I-GRP	 40	 24	 186	 0.625	 0.17699115	 0.27586207
B-CORP	 14	 32	 113	 0.3043478	 0.11023622	 0.16184969
B-GRP	 13	 2	 105	 0.8666667	 0.11016949	 0.19548872
I-CORP	 21	 45	 101	 0.3181818	 0.17213115	 0.22340426
I-LOC	 0	 0	 61	 0.0	 0.0	 0.0
I-PER	 23	 20	 157	 0.53488374	 0.12777779	 0.20627804
I-PROD	 0	 0	 129	 0.0	 0.0	 0.0
I-CW	 6	 37	 155	 0.13953489	 0.03726708	 0.05882353
B-PER	 19	 11	 125	 0.6333333	 0.13194445	 0.21839082
tp: 172 fp: 322 fn: 1

In [104]:
ner_model.stages[1].write().overwrite().save('./NER_bert_20200219')

In [105]:
test_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|[[word_embeddings...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|[[word_embeddings...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|[[word_embeddings...|
|প্রাইমাভেরা ঝুঁকি...|[[document, 0, 64...|[[document, 0, 64...|[[token, 0, 10, প...|[[pos, 0, 10, _, ...|

In [106]:
predictions = ner_model.transform(test_data.select("sentence", "token", "label"))
predictions.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            sentence|               token|               label|                bert|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[[document, 0, 10...|[[token, 0, 4, সম...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 16...|[[token, 0, 6, সং...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|[[document, 0, 11...|[[token, 0, 3, এক...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [107]:
predictions.select('token.result','label.result','ner.result').show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                  result|                                  result|                                  result|
+----------------------------------------+----------------------------------------+----------------------------------------+
|[সমস্ত, বেতন, নিলামের, সাধারণ, ব্যবহা...|[O, O, O, O, O, O, O, O, O, O, B-CORP...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|
|[সংস্করণ, ৩, প্রকাশের, আগে,, গ্রাফিক্...|[O, O, O, O, B-CW, I-CW, I-CW, O, O, ...|[O, O, O, O, B-GRP, I-GRP, I-GRP, I-G...|
|[একটি, পাইলট, (হাটন, দ্বারা, চিত্রিত)...|[O, O, O, O, O, B-LOC, I-LOC, O, O, O...|[O, O, O, O, O, B-PROD, I-PROD, O, O,...|
|[প্রাইমাভেরা, ঝুঁকি, বিশ্লেষণ, -, প্র...|          [B-CW, O, O, O, O, O, O, O, O]|             [O, O, O, O, O, O, O, O, O]|
|[১৮০০, -এর, দশকে, ম্যাটভে, কাজাকভ, দ্...|[O, O, O, B-PER, I-PER, O, O, O, O, O...|[O, O, O, O, B-CORP, O, O, O, O, O, O...|


In [108]:
predictions.printSchema()

root
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (n

In [109]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+----------+------------+----------+
|token     |ground_truth|prediction|
+----------+------------+----------+
|সমস্ত     |O           |O         |
|বেতন      |O           |O         |
|নিলামের   |O           |O         |
|সাধারণ    |O           |O         |
|ব্যবহারিক |O           |O         |
|উদাহরণ    |O           |O         |
|বিভিন্ন   |O           |O         |
|পেনি      |O           |O         |
|নিলাম     |O           |O         |
|/         |O           |O         |
|বিডিং     |B-CORP      |O         |
|ফি        |I-CORP      |O         |
|নিলাম     |I-CORP      |O         |
|ওয়েবসাইটে|O           |O         |
|পাওয়া    |O           |O         |
|যাবে।     |O           |O         |
|সংস্করণ   |O           |O         |
|৩         |O           |O         |
|প্রকাশের  |O           |O         |
|আগে,      |O           |O         |
+----------+------------+----------+
only showing top 20 rows



In [92]:
# !git clone https://github.com/JohnSnowLabs/spark-nlp

In [96]:
# %cd spark-nlp/python/tensorflow
# %cd /content

In [94]:
# %env PYTHONPATH="/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %env PYTHONPATH="$/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %set_env PYTHONPATH=/content/spark-nlp/python/tensorflow/lib/ner:/env/python ## working solution
# %set_env PYTHONPATH=/env/python

In [90]:
# !source /content/ner_venv/bin/activate; python ner/create_models.py 13 128 220 /content/data

In [82]:
# !pip install virtualenv
# !virtualenv ner_venv
# !virtualenv -p /usr/bin/python3.6 ner_venv ## working solution

In [88]:
# !source /content/ner_venv/bin/activate; pip install tensorflow==1.15.0

Reference: [1](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/blogposts/3.NER_with_BERT.ipynb)